In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bfs
from IPython.display import HTML
import pandas as pd
import numpy as np
import time
import random
import os.path
import os
from Crypto.Cipher import AES
import binascii
import json
from ipywidgets import FloatProgress
from IPython.display import display

# Get all acode (runner unique code) in a run event

In [ ]:
HTML('<iframe src=https://services.datasport.com/2015/lauf/transviamala/alfaw.htm width=1000 height=350></iframe>')

In [ ]:
def get_all_acode(url):
    alpha_page = rq.get(url)
    alpha_page_soup = bfs(alpha_page.text, 'html5lib')

    runners = alpha_page_soup.body.findAll('span', attrs={'class': 'myds'})
    
    return {runner['acode']: str.strip(runner.text) for runner in runners}

def get_all_acode_from_run_event(url, file=False):
    
    if file and os.path.isfile(file):
        print('Read acode from file: ' + file)
        return pd.read_csv(file, index_col='acode').to_dict()['name']
    
    data = []
    page = rq.get(url)
    soup = bfs(page.text, 'html5lib')
    table_links = soup.select('font > a[href*=ALF]')
    all_acode = {}
    if url[-1] != '/':
        url += '/'
    for idx, link in enumerate(table_links):
        full_link = url + link['href']
        
        print(str(idx+1) + '/' + str(len(table_links)) + ' - Processing ' + full_link)

        all_acode = {**all_acode, **get_all_acode(full_link)}
        
        time.sleep(random.uniform(0.1, 0.3))

    if file:
        print('Write acode in file: ' + file)
        pd.Series(all_acode).to_csv(file, header=['name'], index_label='acode')
        
    return all_acode

In [ ]:
#acode_list = get_all_acode_from_run_event('https://services.datasport.com/2009/diverse/trophy/')

In [ ]:
#list(acode_list.items())[0:10]

# Get run events

In [ ]:
HTML('<iframe src=https://www.datasport.com/en/Calendar/ width=1000 height=350></iframe>')

## Get all params

In [ ]:
def get_all_params_calendar():
    calendar_page = rq.get('https://www.datasport.com/en/Calendar/')
    calendar_soup = bfs(calendar_page.text, 'html5lib')

    selector_table = calendar_soup.find('table', attrs={'id': 'ds-calendar-header'})

    available_params = {}
    for selector in selector_table.findAll('select'):
        available_params[selector['name']] = {option.text: option['value'] for option in selector.findAll('option')}

    return available_params

In [ ]:
#calendar_available_params = get_all_params_calendar()
#calendar_available_params

## Get all run events url in a calendar

In [ ]:
def get_run_events_url(year=2009, month=1, country='CCH', sport='Running'):

    calendar_params = {
        'dr': '',
        'lastQuery': 'D147BC896417D2D2B96FA1AADD893731',
        'eventsearch': '',
        'eventservice': 'all',
        'start': 1,
        'etyp': sport,
        'eventlocation': country,
        'eventmonth': month,
        'eventyear': year,
    }

    calendar_page = rq.post('https://www.datasport.com/en/Calendar/', data = calendar_params)
    calendar_soup = bfs(calendar_page.text, 'html5lib')

    table = calendar_soup.find('table', attrs={'id': 'ds-calendar-body'})

    all_event_url = {}
    for row in table.findAll('tr'):
        columns = row.findAll('td')
        if len(columns) >= 4:
            url = columns[4].find('a')
            if url:
                all_event_url[url['href']] = {
                    'year': year,
                    'month': month,
                    'country': country,
                    'sport': sport,
                    'full_date': str.strip(columns[0].text),
                    'name': str.strip(columns[1].text),
                }

    return all_event_url

In [ ]:
#get_run_events_url()

## Get all run event urls

In [ ]:
def get_all_run_events():
    run_events = {}
    calendar_available_params = get_all_params_calendar()
    for year_text, year_value in calendar_available_params['eventyear'].items():
        if year_value != 'all':
            for month_text, month_value in calendar_available_params['eventmonth'].items():
                if month_value != 'all':
                    print('Processing: Year ' + year_value + ' / Month ' + month_value)
                    run_events = {**run_events,  **get_run_events_url(year_value, month_value)}
                    time.sleep(random.uniform(0.5, 2))
    return run_events

In [ ]:
#run_events = get_all_run_events()

In [ ]:
#run_events_df = pd.DataFrame(run_events).T
#run_events_df.index.name = 'url'

In [ ]:
#run_events_df.to_csv('Data/run_events.csv')

In [ ]:
!head Data/run_events.csv

In [ ]:
run_events_df = pd.read_csv('Data/run_events.csv')
run_events_df.head()

## Get all acode in all runs from 2009

In [ ]:
#run_events_df_2009_2015 = run_events_df[[year >= 2009 and year < 2016 for year in run_events_df['year']]]
#run_events_df_2009_2015.size

In [ ]:
#run_events_df_2009_2015.to_csv('Data/run_events_2009_2015.csv', index_label='acode_index')
!head Data/run_events_2009_2015.csv

In [ ]:
run_events_df_2009_2015 = pd.read_csv('Data/run_events_2009_2015.csv', index_col='acode_index')

In [ ]:
def write_all_acode_from_list_run_event(run_events, path='Data/acode_2009_2015/'):
    for run_event in run_events.itertuples():
        print('Processing: Run "' + run_event.name + '" / Date ' + run_event.full_date + ' / Url ' + run_event.url)
        get_all_acode_from_run_event(run_event.url, path + str(run_event.Index) + '.csv')

In [ ]:
PATH_TO_DATA = './Data/acode_2009_2015/'

In [ ]:
#all_acode_2009_2015 = get_all_acode_from_list_run_event(run_events_df_2009_2015, PATH_TO_DATA)

In [ ]:
# definir une méthode

# get all data and remove the double
file_list = os.listdir(PATH_TO_DATA)
result_dataframe = pd.read_csv(PATH_TO_DATA + '1000.csv')
for file in file_list:
    pd.concat([result_dataframe, pd.read_csv(PATH_TO_DATA+file)], axis=0)

In [ ]:
result_dataframe.head()

In [ ]:
len(result_dataframe)

In [ ]:
FILE_NAME = 'unique_acode_2009_2015.csv'
#if(len(result_dataframe['acode'].unique()) == len(result_dataframe)):
#    result_dataframe.to_csv(PATH_TO_DATA+FILE_NAME)

# Get runner information

In [ ]:
#####################################
# DECRYPTION OF RUNNING INFORMATION #
#####################################

KEY = '0187fcdf7ac3d90d68334afa03b87efe' # Decode Base64 string + Hex encoder
IV = '33a39433e4dde7c4ddb9d4502b8905d4' # Decode Base64 string + Hex encoder
KEY_BYTES = binascii.a2b_hex(KEY)
IV_BYTES = binascii.a2b_hex(IV)
CIPHER = AES.new(KEY_BYTES, AES.MODE_CBC, IV_BYTES, segment_size=128)

def unpad(string):
    '''
    This function removes the PKCS#7 padding from a text string.
    
    Parameters
    string: data with padding
    '''

    length = len(string)
    pad_size = string[-1]
    return string[:length - pad_size]


def decrypt_data(cipher, encrypted):
    '''
    This function decrypts data given key and iv.
    
    Parameters
    cipher: AES cipher used to decrypt data
    encrypted: encrypted data
    '''

    try:
        decrypted = cipher.decrypt(binascii.a2b_base64(encrypted).rstrip())
    except ValueError:
        return None

    return unpad(decrypted).decode()

In [ ]:
###################################
# HANDLE OF RUNNER'S INFORMATIONS #
###################################

def retrieve_runner_page(url):
    '''
    This function retrieves personal page's content of a runner and session cookies.
    
    Parameters
    url: URL of personal page of a runner
    '''
    
    filters_page = rq.get(url)
    cookies = filters_page.cookies.get_dict()
    page = bfs(filters_page.text, 'html.parser')
    return page, cookies


def retrieve_birthyear_location(page):
    '''
    This function retrieves birthyear and location of a runner in his personal page.
    
    Parameters
    page: HTML page of a runner
    '''
    
    small_p = page.findAll('p', { 'class': 'small' })
    birthyear = small_p[1].text.split(" ")[1:][0]
    location = small_p[0].text.strip()
    return birthyear, location


def parse_html_runner_table(table, base_url='https://www.datasport.com'):
    '''
    This function transforms HTML table containing all runnings of a given runner into a pandas DataFrame.
    
    Parameters
    table: HTML table to be parsed
    base_url: root URL of website (by default, https://www.datasport.com)
    '''
    
    n_columns = 0
    n_rows=0
    column_names = []

    # First, we find number of rows and columns, but also column titles if we can
    for row in table.find_all('tr'):

        # We determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows += 1
            if n_columns == 0:
                # We set the number of columns for our table
                n_columns = len(td_tags)

        # If we find column names, we store them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())

    # We raise an exception if there is a problem with columns' structure
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    # We create pandas DataFrame according to HTML table structure
    columns = column_names if len(column_names) > 0 else range(0, n_columns)
    df = pd.DataFrame(columns = columns, index= range(0, n_rows))

    url_information_runner = []
    row_marker = 0
    
    # We retrieve all data of table
    for row in table.find_all('tr'):
        column_marker = 0

        columns = row.find_all('td')
        for index, column in enumerate(columns):
            df.iat[row_marker,column_marker] = column.get_text()

            # We add URL to get information about the runners
            if(index == 1):
                url_information_runner.append(base_url + column.find('a')['href'])

            column_marker += 1
            
        if len(columns) > 0:
            row_marker += 1

    # We convert columns to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    # We finally add URL of running event in the DataFrame
    df['url_run_event'] = pd.Series(url_information_runner)

    return df


def retrieve_time_table(page):
    '''
    This function retrieves the time table (i.e. all runnings of a runner) in a given page.
    
    Parameters
    page: HTML page to be parsed in order to recover the time table
    '''
    
    table_run_event = page.find('table', {'id': 'timeTable'})
    return parse_html_runner_table(table_run_event)


def retrieve_running_information(information_runner, running_number, cookies):
    '''
    This function retrieves all the information of a running, given a runner.
    
    Parameters
    information_runner: Information of the considered runner
    running_number: Number associated to the running for which we want to recover information
    cookies: Session cookies to use with GET request
    '''
    
    time.sleep(random.uniform(0.5, 1))
    ajax_response = rq.get(information_runner.iloc[[running_number]]['url_run_event'].values[0], cookies=cookies)
    cipher = AES.new(KEY_BYTES, AES.MODE_CBC, IV_BYTES, segment_size=128)
    return decrypt_data(cipher=cipher, encrypted=ajax_response.text)


def force_retrieve(url, running_number):
    '''
    This function forces the retrieve of the running information using a new session.
    
    Parameters
    url: URL of the personal page of runner
    running_number: Number associated to the running for which we want to recover information
    '''
    
    # We allow at most three (3) attempts
    for i in range(0, 3):
        page, cookies = retrieve_runner_page(url)
        information_runner = retrieve_time_table(page)
        decrypted_response = retrieve_running_information(information_runner, running_number, cookies)
        if decrypted_response != None:
            break
    return decrypted_response

In [ ]:
########################################
# RETRIEVE OF ALL RUNNERS' INFORMATION #
########################################

def get_runners_information(
    acode_file='./Data/acode_2009_2015/unique_acode_2009_2015.csv',
    base_url='https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode=',
    store_runners_information_file='./Data/runners.csv',
    store_runs_information_file='./Data/runs.csv'):

    data_acode = pd.read_csv(acode_file)
    
    data_runners = []
    data_runs = []

    print('Number of runners: ' + str(len(data_acode)))
    
    for idx_acode, row_acode in data_acode.iterrows():
        
        url = base_url + row_acode['acode']
        
        print('Runner acode ' + row_acode['acode'], end='')
        
        # We retrieve personal page of the runner and store cookies for further calls to server
        page, cookies = retrieve_runner_page(url)

        # We get the birthyear, location and all runnings associated to the runner
        birthyear, location = retrieve_birthyear_location(page)
        information_runner = retrieve_time_table(page)
        
        # We store personal information of the runner
        data_runners.append({
                'acode': row_acode['acode'],
                'name': row_acode['name'],
                'birthyear': birthyear,
                'location': location
            })

        # We retrieve all runs of the runner
        print(' / Number of runs: ' + str(len(information_runner)))
        progress_bar = FloatProgress(min=0, max=len(information_runner))
        display(progress_bar)

        for idx_runner, row_runner in information_runner.iterrows():
            
            decrypted_response = retrieve_running_information(information_runner, idx_runner, cookies)

            # We force retrieve using new session if decrypt process failed
            if decrypted_response == None:
                decrypted_response = force_retrieve(url, idx_runner)
           
            progress_bar.value += 1
            
            if decrypted_response == None:
                print('FATAL ERROR')
                print(row_acode['acode'])
                print(idx_runner)
                print(row_runner['url_run_event'])
                print(url)
                print('**********')
                continue

            running_information = json.loads(decrypted_response)
            
            data_runs.append({**running_information, 'acode': row_acode['acode']})
        
        # END OF LOOP ON RUNNINGS
        
        progress_bar.close()
        
    # END OF LOOP ON RUNNERS

    df_data_runners = pd.DataFrame(data_runners)
    df_data_runs = pd.DataFrame(data_runs)
    
    if store_runners_information_file:
        df_data_runners.to_csv(store_runners_information_file, index=False)
    if store_runs_information_file:
        df_data_runs.to_csv(store_runs_information_file, index=False)

    return [df_data_runners, df_data_runs]

In [ ]:
df_data_runners, df_data_runs = get_runners_information()